## Writing to google sheet #########
from __future__ import print_function

import googleapiclient.discovery as discovery
from httplib2 import Http
from oauth2client import client
from oauth2client import file
from oauth2client import tools

import pygsheets
import pandas as pd

import time
import json
import sys
import random
import requests

#authorization
gc = pygsheets.authorize(service_file= path_docs + 'credentials2.json')

#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sh = gc.open('telegram monitoring')

#select the first sheet 
wks = sh[0]

#Get the data from the Sheet into python as DF
read = wks.get_as_df()
#Print the head of the datframe
read.head()

app_start = read.iloc[0]['Values']
print(app_start)
search_term = read.iloc[2]['Values']
print(search_term)

def slack_msg(message):    
    title = (f"telegram_updates")
    slack_data = {
        "username": "telegram_monitoring",
        "icon_emoji": ":satellite:",
        #"channel" : "#somerandomcahnnel",
        "attachments": [
            {
                "color": "#9733EE",
                "fields": [
                    {
                        "title": title,
                        "value": message,
                        "short": "false",
                    }
                ]
            }
        ]
    }
    byte_length = str(sys.getsizeof(slack_data))
    headers = {'Content-Type': "application/json", 'Content-Length': byte_length}
    response = requests.post(url, data=json.dumps(slack_data), headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

if app_start == "Yes":

    ####Loading all the telegram packages
    import configparser
    from telethon import TelegramClient
    from telethon.errors import SessionPasswordNeededError
    from telethon.sync import TelegramClient
    from telethon import functions, types

    from telethon.tl.functions.channels import JoinChannelRequest
    from telethon.errors.rpcerrorlist import FloodWaitError
    import asyncio

    ##Laoding the other required packages
    import re
    import os
    import pandas as pd

    #TEXT EXTRACTION FROM IMAGES
    # Loading all the required packages
    from matplotlib import pyplot as plt
    from easyocr import Reader ###Pretrained text extraction model from an AI company JaidedAI
    import argparse
    import cv2
    import glob
    import numpy as np

    from IPython.display import clear_output
    import time 

    ###TEXT ANALYSIS NLP Packages
    ## for data
    import pandas as pd
    import collections
    import json
    ## for plotting
    import matplotlib.pyplot as plt
    import seaborn as sns
    #import wordcloud
    ## for text processing
    import re
    import nltk
    ## for language detection
    import langdetect 
    ## for sentiment
    #from textblob import TextBlob
    ## for ner
    #import spacy
    ## for vectorizer
    from sklearn import feature_extraction, manifold
    ## for word embedding
    #import gensim.downloader as gensim_api
    ## for topic modeling
    #import gensim
    import os

    import json
    import asyncio
    from datetime import date, datetime

    from telethon import TelegramClient
    from telethon.errors import SessionPasswordNeededError
    from telethon.tl.functions.messages import (GetHistoryRequest)
    from telethon.tl.types import (
        PeerChannel
    )

    from IPython.display import clear_output
    import time 
    import shutil

    ###Enter telegram channels credentials to access the telegram API
    api_id = ""
    api_hash = ''
    # use full phone number including + and country code
    phone = ''
    username = ''
    
    client = TelegramClient(username, api_id, api_hash)
    result = 'global'

    ###Default code to cut the long running functions
    import nest_asyncio
    nest_asyncio.apply()

    print("Welcome to the Telegram monitoring application")
    async def main():
        await client.start()
        search = search_term
        global result

        result = await client(functions.contacts.SearchRequest(
            q=search,
            limit=100
            ))
    async with client:
        client.loop.run_until_complete(main())

    ###Data Manipulation of the data obtained above

    strr = result.stringify()
    strr1 = strr.split("chats")[1]
    trs = re.findall('Channel(.*)', strr1)
    a = [ False if 'Forbidden' in a else True for a in trs ]

    title = re.findall('title=(.*),', strr)
    channel_id = re.findall('channel_id=(.*)', strr)
    chn_name = re.findall('username=(.*),', strr1)
    chn_name1 = [x.replace("'", "") for x in chn_name  ]
    chn_name1 = [x.replace(",", "") for x in chn_name1  ]
    
    print(chn_name1)
    import copy
    chn_name2 = copy.deepcopy(chn_name1)
    
    print(chn_name2)

    for x in range(0, len(a)):
        if a[x] != True:
            chn_name2.insert(x,'None')

    dtf_c = pd.DataFrame(
        {'title': title,
         'channelid': channel_id,
         'username' : chn_name2
        }
    )

    print("Channels found" + "\n" + "\n")
    print(dtf_c)        
    
    #open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
    sh = gc.open('telegram monitoring')

    #select the first sheet 
    wks = sh[1]

    #update the first sheet with df, starting at cell B2. 
    wks.set_dataframe(dtf_c,(1,1))
    print("Joining the channels before scrapping" + "\n" )
    
    for x in range(0,len(chn_name1)):
        if 'None' in chn_name1:
            chn_name1.remove("None")
        #chn_name1

    ####TO JOIN THE GROUPS ########
    
    #chn_name1 = [x.remove("None") for x in chn_name1  ]
    CHANNELS = chn_name1 # the channels you want to join
    
    print(CHANNELS)

    async def main():
        async with client:
        #async with TelegramClient(username, api_id, api_hash) as client:
            for channel in CHANNELS:
                try:
                    await client(JoinChannelRequest(channel))
                    print("Joining channel  " + channel)
                except FloodWaitError as fwe:
                    print(f'Waiting for {fwe}')
                    await asyncio.sleep(delay=fwe.seconds)

    asyncio.run(main())    

    print("Channel searched and joined for the search term")
    print("\n" + "Waiting for 30 seconds")
    time.sleep(1)
    
    channel_scrape = read.iloc[4]['Values']
    print(channel_scrape)
    channel_name = read.iloc[6]['Values']
    print(channel_name)

    if channel_scrape == 'Yes':
        
        source_path = ''

        path = source_path + channel_name
        if os.path.exists(path):
            shutil.rmtree(path)
        
        if not os.path.exists(path):
            os.makedirs(path)    

        ## Code to scrape the messages from a channel passed
        # some functions to parse json date
        class DateTimeEncoder(json.JSONEncoder):
            def default(self, o):
                if isinstance(o, datetime):
                    return o.isoformat()

                if isinstance(o, bytes):
                    return list(o)

                return json.JSONEncoder.default(self, o)

        # Create the client and connect
        async def main(phone):
            #await client.start()
            print("Client Created")
            # Ensure you're authorized
            global my_channel
            my_channel = channel_name
            print("\n" + "Scrapping message for the channel  " + my_channel)

            offset_id = 0
            limit = 100
            all_messages = []
            total_messages = 0
            total_count_limit = 0

            while True:
                print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
                history = await client(GetHistoryRequest(
                    peer=my_channel,
                    offset_id=offset_id,
                    offset_date=None,
                    add_offset=0,
                    limit=limit,
                    max_id=0,
                    min_id=0,
                    hash=0
                ))
                if not history.messages:
                    break
                messages = history.messages
                for message in messages:
                    all_messages.append(message.to_dict())
                offset_id = messages[len(messages) - 1].id
                total_messages = len(all_messages)
                if total_count_limit != 0 and total_messages >= total_count_limit:
                    break

            with open(channel_name + '.json', 'w') as outfile:
                json.dump(all_messages, outfile, cls=DateTimeEncoder)

        async with client:
            client.loop.run_until_complete(main(phone))

        ###Storing the scrapped  channel content in a variable
        import pandas as pd
        df_txt_msgs = pd.read_json(channel_name + '.json')
        print("\n" + "Downloading images from "  + channel_name)

        async def run():
            await client.connect()
            channel = await client.get_entity(channel_name)
            i = 0
            global path1

            path1 = path + "/images/"
            async for message in client.iter_messages(channel):
                i = i + 1

                clear_output(wait=True)
                print("\n" +  "Number of messages  " + str(i))
                await client.download_media(message, file = path1 )

        client.loop.run_until_complete(run())    

        print("\n" + "Image to text scraping started" + "\n")

        langs = ['en']
        reader = Reader(langs, gpu=0)

        ###Reading text from Image ####
        def plt_imshow(title, image):
            # convert the image frame BGR to RGB color space and display it
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.imshow(image)
            plt.title(title)
            plt.grid(False)
            plt.show()

        def cleanup_text(text):
            # strip out non-ASCII text so we can draw the text on the image
            # using OpenCV
            return "".join([c if ord(c) < 128 else "" for c in text]).strip()

        # load the input image from disk
        cnt = 0
        img_nme = []
        pix_perc = []

        txtt = []
        na_i = []
        a = 1
        pr1 = 0
        
        import os
        listt = os.listdir(path) # dir is your directory path
        number_files = len(listt)
        
        print(number_files)
        if number_files > 1:
            # iterate through list of files in a directory
            for images in glob.iglob(path1  + '*.jpg'):
                try:
                    txt = []
                    # load the input image from disk
                    image = cv2.imread(images)
                    img_name = images

                    # OCR the input image using EasyOCR
                    print("\n" + "Extracting text from image..." + str(a))

                    #reader = Reader(langs, gpu=0)
                    results = reader.readtext(image)
                    ######Calculating the text pixels in the image
                    if(len(results) != 0):
                        #Capturing the results values in 3 different variables - bbox: Pixel values, text : Actual text content, Prob : Probability
                        for (bbox, text, prob) in results:
                            # cleanup the text and draw the box surrounding the text along
                            # with the OCR'd text itself
                            text = cleanup_text(text)
                            txt.append(text)

                    txtt.append(txt)
                    images = images.replace(path + "/images/","")
                    img_nme.append(images)

                    a = a+1
                    global dtf

                    dtf = pd.DataFrame(list(zip(img_nme,  txtt )), 
                        columns =['Image',  'text']) 

                    #dtf.to_excel(path + '/'+ 'image_extraction_data.xlsx')
                    #print(dtf)
                except:
                    na_i.append(images)

            print("\n" + "Text scrapping completed" + "\n")
            print("\n" + "Text Analysis started" + "\n")
            
            def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
                ## clean (convert to lowercase and remove punctuations and characters and then strip)
                text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

                ## Tokenize (convert from string to list)
                lst_text = text.split()
                ## remove Stopwords
                if lst_stopwords is not None:
                    lst_text = [word for word in lst_text if word not in 
                                lst_stopwords]

                ## Stemming (remove -ing, -ly, ...)
                if flg_stemm == True:
                    lst_text = [ps.stem(word) for word in lst_text]

                ## Lemmatisation (convert the word into root word)
                if flg_lemm == True:
                    lem = nltk.stem.wordnet.WordNetLemmatizer()
                    lst_text = [lem.lemmatize(word) for word in lst_text]

                ## back to string from list
                text = " ".join(lst_text)
                return text
            dtf["text_clean"] = dtf["text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords = None))

            ####Finding the count of following words
            lst_words = []
            ## count
            lst_grams = [len(word.split(" ")) for word in lst_words]
            vectorizer = feature_extraction.text.CountVectorizer(
                             vocabulary=lst_words, 
                             ngram_range=(min(lst_grams),max(lst_grams)))
            dtf_X = pd.DataFrame(vectorizer.fit_transform(dtf["text_clean"]).todense(), columns=lst_words)
            ## add the new features as columns
            dtf = pd.concat([dtf, dtf_X.set_index(dtf.index)], axis=1)
            dtf.head()

            dtf.to_excel(path + "/" + "Image_text_output.xlsx")
            print("\n" + "Text Analysis completed" + "\n")
            print("Check for the output at" + path)
            
            df_txt_msgs["text_clean"] = df_txt_msgs["message"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords = None))

            ####Finding the count of following words
            lst_words = []
            ## count
            lst_grams = [len(word.split(" ")) for word in lst_words]
            vectorizer = feature_extraction.text.CountVectorizer(
                             vocabulary=lst_words, 
                             ngram_range=(min(lst_grams),max(lst_grams)))
            dtf_X = pd.DataFrame(vectorizer.fit_transform(df_txt_msgs["text_clean"]).todense(), columns=lst_words)
            ## add the new features as columns
            df_txt_msgs = pd.concat([df_txt_msgs, dtf_X.set_index(df_txt_msgs.index)], axis=1)
            df_txt_msgs.head()

            #df_txt_msgs.to_excel(path + "/" + "Text_messages_output.xlsx")
            df_txt_msgs.to_csv(path + "/" + "Text_messages_output.csv")
            
            dtf_invoice = dtf.loc[dtf[''] == 1]
            dtf_invoice_list = list(dtf_invoice['Image'])
            dtf_invoice_list

            
            if not os.path.exists(path2):
                os.makedirs(path2)    

            os.chdir(path1)
            for a in glob.iglob('*.jpg'):
                if a in dtf_invoice_list:
                    shutil.copy(a, path2)
                    print(a)

            print("\n" + "Text Analysis completed" + "\n")
            print("Check for the output at" + path)

        import shutil
        shutil.make_archive(path , 'zip', path)

        file_name = path   + ".zip"
        print(file_name)

        import os
        from slack_sdk import WebClient
        from slack_sdk.errors import SlackApiError

        SLACK_BOT_TOKEN_ = ''

        client = WebClient(SLACK_BOT_TOKEN_)

        try:
            filepath=file_name
            response = client.files_upload(channels='app-telegram-monitoring', file=file_name)
            assert response["file"]  # the uploaded file
        except SlackApiError as e:
            # You will get a SlackApiError if "ok" is False
            assert e.response["ok"] is False
            assert e.response["error"]  # str like 'invalid_auth', 'channel_not_found'
            print(f"Got an error: {e.response['error']}")    

    else:
        print("Scrapping turned off")

else:
    print("Telegram App turned off")
    